In [2]:
"""
    Objective 1: Lab data
        - Plot the data
        - Check average performance (binnned) increases w/ delta -> RM-ANOVA
        - Both for orientation and color
    Objective 2: mturk data
        - Bootstrap samples to compute standard error
        - relevant data: check average performance (binned) increases w/ delta -> RM-ANOVA
        - Irrelevant data: check average performance (binned) is 
            significantly different from chance -> one-sided t-test
    Objective 3: Lab vs. mturk data (relevant)
        - Check mturk data is significanlty worse than lab data -> two-sided t-test
"""
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib

#1. Laboratory experiment
- Change localization w/ one orientation change and one color change
- N = 5

##1.1 Load files

In [17]:
# to load multiple files, let's use unix-style wildcards
import fnmatch # file name matching
import glob # path name matching
import os
import io

pathname = 'one_change_lab'

a = os.listdir('one_change_lab')

#sp.io.loadmat

#HJK6_6_2_150_20130618T114201.mat
#print os.path.dirname(pathname)

#print os.path.exists("/home/")
#for file in os.listdir(pathname):
#   if fnmatch.fnmatch(file, '*.mat'):
#      print file


#import sys
#sys.path.insert(0, 'hongsup/Dropbox/Public/change localization/conjunction/output/') #exp6/
#mat = sp.io.loadmat('hongsup/Dropbox/Public/change localization/conjunction/output/exp6/HJK6_6_2_150_20130618T114201.mat')

In [ ]:
print os.listdir